In [2]:
import pandas as pd
import duckdb

# Questions to Answer:
- DAU/MAU
- Total page views
- Page View by Page
- Referrels
- Devices
- Search terms
- num search to num results
- CTRs
- sponsored vs organic impressions
- total impressions
- business page views
- business sign ups
- Paid businesses

In [3]:
df = pd.read_csv('ccbd-data.csv')
df = df[df['originalTimestamp'] >= '2025-01-01']
print(df.shape)
print(df.columns)
df.head()

(9761, 44)
Index(['anonymousId', 'messageId', 'userId', 'originalTimestamp', 'userAgent',
       'page.path', 'page.referrer', 'page.search', 'page.title', 'page.url',
       'userAgentData.brands', 'userAgentData.mobile',
       'userAgentData.platform', 'business_id', 'sponsored_listing',
       'category', 'name', 'path', 'referrer', 'search', 'title', 'url',
       'category_properties', 'name_properties', 'button_name', 'location',
       'city', 'search_text', 'parish', 'traits', 'input_num', 'input_name',
       'email', 'input_value', 'address', 'other_category', 'description',
       'display_email', 'business_name', 'main_category', 'phone',
       'sub_category', 'website', 'event_type'],
      dtype='object')


,anonymousId,messageId,userId,originalTimestamp,userAgent,page.path,page.referrer,page.search,page.title,page.url,...,address,other_category,description,display_email,business_name,main_category,phone,sub_category,website,event_type
0,e484246f-8684-4f92-ab30-1d093a3f5f60,ajs-next-1745859266475-8684ff92-2b30-4d09-ba3f...,NaN,2025-04-28 16:54:26.475000+00:00,Mozilla/5.0 (Linux; Android 12; Pixel 6 Build/...,/,https://www.facebook.com/,?fbclid=IwZXh0bgNhZW0CMTEAAR58oxCxVDihSoLzwMP_...,Colorado Catholic Business Directory,https://coloradocatholicbusinessdirectory.com/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,impression
1,e484246f-8684-4f92-ab30-1d093a3f5f60,ajs-next-1745859265303-246f8684-ff92-4b30-9d09...,NaN,2025-04-28 16:54:25.303000+00:00,Mozilla/5.0 (Linux; Android 12; Pixel 6 Build/...,/,https://www.facebook.com/,?fbclid=IwZXh0bgNhZW0CMTEAAR58oxCxVDihSoLzwMP_...,Colorado Catholic Business Directory,https://coloradocatholicbusinessdirectory.com/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,impression
2,e484246f-8684-4f92-ab30-1d093a3f5f60,ajs-next-1745859265295-13e48424-6f86-44ff-922b...,NaN,2025-04-28 16:54:25.295000+00:00,Mozilla/5.0 (Linux; Android 12; Pixel 6 Build/...,/,https://www.facebook.com/,?fbclid=IwZXh0bgNhZW0CMTEAAR58oxCxVDihSoLzwMP_...,Colorado Catholic Business Directory,https://coloradocatholicbusinessdirectory.com/...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,page
3,39b9e148-9871-4dae-8fe9-b1f5845d3932,ajs-next-1745860775776-e2d11c37-72f4-449e-bbf8...,1743547621069x279645472884266780,2025-04-28 17:19:35.776000+00:00,Mozilla/5.0 (iPhone; CPU iPhone OS 18_4_1 like...,/,https://bit.ly/,NaN,Colorado Catholic Business Directory,https://coloradocatholicbusinessdirectory.com/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,impression
4,39b9e148-9871-4dae-8fe9-b1f5845d3932,ajs-next-1745860775813-1c3772f4-d49e-4bf8-b69d...,1743547621069x279645472884266780,2025-04-28 17:19:35.813000+00:00,Mozilla/5.0 (iPhone; CPU iPhone OS 18_4_1 like...,/,https://bit.ly/,NaN,Colorado Catholic Business Directory,https://coloradocatholicbusinessdirectory.com/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,impression


In [4]:
df['event_type'].value_counts()

event_type
impression       4207
page             2856
button_click     1240
identify          516
input_changed     487
search            381
db_update          84
Name: count, dtype: int64

In [5]:
query = """
with base as (
    SELECT DATE(originalTimestamp) as the_date, COALESCE(userId, anonymousId) as user_id
    FROM df 
)

select the_date, count(distinct user_id) as dau 
from base
group by the_date
order by the_date
"""
dau = duckdb.sql(query).df()
print(dau.shape)
dau.head()

(87, 2)


,the_date,dau
0,2023-06-11,1
1,2025-04-28,2
2,2025-04-29,18
3,2025-04-30,8
4,2025-05-01,12


In [6]:
query = """
with base as (
    SELECT DATE_TRUNC('month', DATE(originalTimestamp)) as month_year, 
           COALESCE(userId, anonymousId) as user_id
    FROM df 
)

select month_year, count(distinct user_id) as monthly_active_users
from base
group by month_year
order by month_year
"""
mau = duckdb.sql(query).df()
print(mau.shape)
mau.head()

(5, 2)


,month_year,monthly_active_users
0,2023-06-01,1
1,2025-04-01,26
2,2025-05-01,415
3,2025-06-01,183
4,2025-07-01,167


In [7]:
query = """
SELECT DATE(originalTimestamp) as the_date, count(distinct messageId) as page_views
FROM df 
WHERE event_type = 'page'
GROUP BY DATE(originalTimestamp)
"""
pvs = duckdb.sql(query).df()
print(pvs.shape)
pvs.head()

(87, 2)


,the_date,page_views
0,2025-05-19,75
1,2025-06-01,10
2,2025-06-05,13
3,2025-06-08,12
4,2025-06-14,9


In [8]:
query = """
with base as (
    SELECT DATE(originalTimestamp) as the_date, 
           COALESCE(userId, anonymousId) as user_id,
           COUNT(DISTINCT messageId) as user_page_views
    FROM df 
    WHERE event_type = 'page'
    GROUP BY DATE(originalTimestamp), COALESCE(userId, anonymousId)
)

SELECT the_date, AVG(user_page_views) as avg_page_views_per_user
FROM base
GROUP BY the_date
ORDER BY the_date
"""
page_views_per_user = duckdb.sql(query).df()
print(page_views_per_user.shape)
page_views_per_user.head()

(87, 2)


,the_date,avg_page_views_per_user
0,2023-06-11,1.000000
1,2025-04-28,1.000000
2,2025-04-29,2.222222
3,2025-04-30,1.125000
4,2025-05-01,2.666667


In [9]:
query = """
SELECT "page.referrer" as referrer, count(*) as count
FROM df 
WHERE "page.referrer" is not null
AND event_type = 'page'
GROUP BY "page.referrer"
ORDER BY count DESC
"""
referrers = duckdb.sql(query).df()
print(referrers.shape)
referrers.head()

(15, 2)


,referrer,count
0,https://coloradocatholicbusinessdirectory.com/,1748
1,https://coloradocatholics.biz/,236
2,https://www.google.com/,56
3,https://www.facebook.com/,14
4,https://bit.ly/,8


In [10]:
query = """
SELECT 
    "userAgentData.platform" as platform,
    count(*) as count
FROM df 
WHERE "userAgentData.platform" is not null
AND event_type = 'page'
GROUP BY "userAgentData.platform"
ORDER BY count DESC
"""
platform_mobile = duckdb.sql(query).df()
print(platform_mobile.shape)
platform_mobile.head(10)

(6, 2)


,platform,count
0,Windows,912
1,macOS,348
2,Android,226
3,Linux,31
4,Chrome OS,18
5,iOS,2


In [12]:
query = """
select event_type, count(*) as count
from df
group by event_type
order by count desc
"""
event_type = duckdb.sql(query).df()
print(event_type.shape)
event_type.head(10)

(7, 2)


,event_type,count
0,impression,4207
1,page,2856
2,button_click,1240
3,identify,516
4,input_changed,487
5,search,381
6,db_update,84


In [28]:
query = """
with impressions as (
    SELECT date(originalTimestamp) as the_date, 
    COALESCE(userId, anonymousId) as user_id, 
    sponsored_listing,
    unnest(string_split(business_id, ', ')) as single_business_id
    FROM df
    WHERE event_type = 'impression'
    and sponsored_listing = 'False'
)

, imp_agg as (
select the_date, user_id, count(distinct single_business_id) as business_count
from impressions
group by the_date, user_id
order by the_date, user_id
)

select the_date, avg(business_count) as avg_business_count
from imp_agg
group by the_date
order by the_date
"""
search_results = duckdb.sql(query).df()
print(search_results.shape)
search_results.head(10)

(73, 2)


,the_date,avg_business_count
0,2023-06-11,58.000000
1,2025-04-29,28.000000
2,2025-04-30,28.000000
3,2025-05-01,28.000000
4,2025-05-04,28.000000
5,2025-05-05,28.666667
6,2025-05-06,29.600000
7,2025-05-07,30.714286
8,2025-05-08,32.000000
9,2025-05-09,32.000000


In [34]:
query = """
with impressions as (
    SELECT date(originalTimestamp) as the_date, 
    COALESCE(userId, anonymousId) as user_id, 
    sponsored_listing,
    unnest(string_split(business_id, ', ')) as single_business_id
    FROM df
    WHERE event_type = 'impression'
)

, imp_agg as (
select the_date, single_business_id, sponsored_listing, count(distinct user_id) as num_impressions
from impressions
group by the_date, single_business_id, sponsored_listing
order by the_date, single_business_id, sponsored_listing
)

select the_date, 
avg(case when sponsored_listing = 'False' then num_impressions else null end) as avg_impressions_non_sponsored,
avg(case when sponsored_listing = 'True' then num_impressions else null end) as avg_impressions_sponsored
from imp_agg
group by the_date
order by the_date
"""
impressions = duckdb.sql(query).df()
print(impressions.shape)
impressions.head(10)

(85, 3)


,the_date,avg_impressions_non_sponsored,avg_impressions_sponsored
0,2023-06-11,1.000000,1.000000
1,2025-04-28,NaN,2.000000
2,2025-04-29,2.000000,17.333333
3,2025-04-30,2.000000,6.666667
4,2025-05-01,3.000000,11.333333
5,2025-05-03,NaN,7.000000
6,2025-05-04,3.000000,4.333333
7,2025-05-05,2.965517,10.000000
8,2025-05-06,4.933333,13.666667
9,2025-05-07,6.718750,16.333333


In [38]:
query = """
with t1 as (
    SELECT date(originalTimestamp) as the_date, business_id, count(distinct messageId) as page_views
    FROM df 
    WHERE event_type = 'page'
    and business_id is not null
    group by date(originalTimestamp), business_id
)

select the_date, avg(page_views) as avg_page_views
from t1
group by the_date
order by the_date
"""
page_views = duckdb.sql(query).df()
print(page_views.shape)
page_views.head(10)

(73, 2)


,the_date,avg_page_views
0,2025-04-29,1.000000
1,2025-05-01,1.166667
2,2025-05-04,1.384615
3,2025-05-05,2.200000
4,2025-05-06,1.444444
5,2025-05-07,3.000000
6,2025-05-08,1.000000
7,2025-05-09,2.642857
8,2025-05-10,2.812500
9,2025-05-11,1.500000


In [6]:
query = """
with t1 as (
    SELECT date(originalTimestamp) as the_date, business_id, count(distinct messageId) as button_clicks
    FROM df 
    WHERE event_type = 'button_click'
    and button_name = 'visit_website'
    group by date(originalTimestamp), business_id
)

select the_date, avg(button_clicks) as avg_button_clicks
from t1
group by the_date
order by the_date
"""
button_clicks = duckdb.sql(query).df()
print(button_clicks.shape)
button_clicks.head(10)

(52, 2)


,the_date,avg_button_clicks
0,2025-05-01,1.000000
1,2025-05-04,1.000000
2,2025-05-05,1.000000
3,2025-05-06,1.000000
4,2025-05-07,1.000000
5,2025-05-09,1.375000
6,2025-05-10,1.000000
7,2025-05-11,1.500000
8,2025-05-12,1.000000
9,2025-05-13,1.636364


In [ ]:
query = """
select DATE_TRUNC('month', DATE(originalTimestamp)) as month_year, count(distinct userId) as signups
from df
where event_type = 'button_click'
and button_name = 'faith'
group by DATE_TRUNC('month', DATE(originalTimestamp))
order by month_year
"""
signups = duckdb.sql(query).df()
print(signups.shape)
signups.head(10)

(3, 2)


,month_year,signups
0,2025-05-01,20
1,2025-06-01,11
2,2025-07-01,8
